In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, WebDriverException
import time
from parsel import Selector
import csv

# 1 st Technique

In [11]:
url = 'https://scholar.google.co.uk/citations?view_op=view_org&hl=en&org=9117984065169182779'
   
options = Options()
options.add_argument("start-maximized")
options.add_argument("disable-infobars")
options.add_argument("--disable-extensions")
driver = webdriver.Chrome( executable_path='C:/Drivers/chromedriver.exe')
driver.get(url)
driver.set_page_load_timeout(5)

In [12]:
paper_descriptions = []
authors = []
titles = []
pubdate = []

paper_count = 1
continuer = 0

breakers = ''
breaker = ''
while True:
    try:
        num_of_auth = driver.find_elements_by_xpath('//*[@id="gsc_sa_ccl"]/div/div/div/h3/a')                      
        for i in range(0, len(num_of_auth)):

            # Enter profiles
            xPathname = '//*[@id="gsc_sa_ccl"]/div/div/div/h3/a'
            names = driver.find_elements_by_xpath(xPathname)[i]
            names.click()
            breakers=''
            while True:
                time.sleep(1)
                if driver.find_element_by_xpath('//*[@id="gsc_a_nn"]').text != breakers:
                    breakers = driver.find_element_by_xpath('//*[@id="gsc_a_nn"]').text
                    driver.find_element_by_xpath('//*[@id="gsc_bpf_more"]/span/span[2]').click()
                else:
                    break
         
            x = driver.find_elements_by_xpath('//*[@id="gsc_a_b"]/tr/td[1]/a')

            time.sleep(1)
            
            # Getting info from each paper loop
            for i in range(0, len(x)):
    
                # Click papers  
                xPath_paper_click = '//*[@id="gsc_a_b"]/tr/td[1]/a'
                papers = driver.find_elements_by_xpath(xPath_paper_click)[i]
                time.sleep(1)
                papers.click()
                
                # Get paper title
                xPath_paper_title = '//*[@id="gsc_vcd_title"]'
                time.sleep(1)
                paper_titles = driver.find_element_by_xpath(xPath_paper_title)
                titles.append(paper_titles.text)
                
                body = driver.find_element_by_xpath('//*[@id="gsc_vcd_table"]')
                
                #Get authors
                if 'Authors' in body.text:
                    time.sleep(0.5)
                    author = driver.find_element_by_class_name('gsc_vcd_value')
                    authors.append(author.text)
                else:
                    author = 'N/A'
                    authors.append(author)

                # Get date
                if 'date'in body.text:
                    time.sleep(0.5)
                    date = driver.find_element_by_xpath('//*[@id="gsc_vcd_table"]/div[2]/div[2]')
                    pubdate.append(date.text)
                else:
                    date ='N/A'
                    pubdate.append(date)
                
                # Get description

                if 'Description' in body.text:
                    time.sleep(0.5)
                    xPath_description = '//*[@id="gsc_vcd_descr"]'
                    description = driver.find_element_by_xpath(xPath_description)
                    paper_descriptions.append(description.text)
                else:
                    description = 'N/A'
                    paper_descriptions.append(description)
                print('paper ' +str(i)+' is ok')                   
                
                driver.back()
                paper_count += 1
            paper_count = 1     
            # resetting the paper_count so it can begin again with the 1st paper after the for loop is complete

            continuer = 1
            driver.back()
            
            time.sleep(1)
        next_button = driver.find_element_by_xpath('//*[@id="gsc_authors_bottom_pag"]/div/button[2]')
        next_button.click()

    except Exception as e:
        print('processed')
        break

paper 0 is ok
paper 1 is ok
paper 2 is ok
paper 3 is ok
paper 4 is ok
paper 5 is ok
paper 6 is ok


KeyboardInterrupt: 

In [10]:
allpapers = []
for i in range(0, len(titles)-1):
    allpapers.append({'Paper_titles': titles[i],
                      'Authors': authors[i],
                      'date': pubdate[i],
                      'Description': paper_descriptions[i]})
    
allpapers

[{'Paper_titles': 'Sonochemistry',
  'Authors': 'Timothy James Mason, J Phillip Lorimer',
  'date': 'Timothy James Mason, J Phillip Lorimer',
  'Description': 'N/A'},
 {'Paper_titles': 'Applied sonochemistry: the uses of power ultrasound in chemistry and processing',
  'Authors': 'Timothy J Mason, J Phillip Lorimer',
  'date': 'Timothy J Mason, J Phillip Lorimer',
  'Description': 'Power ultrasound has been used for many years in two specific industrial areas: cleaning and plastic welding. Over the last ten years an increasing interest has been shown in its potential for use over a much wider range of chemistry and processing which has been grouped together under the general title of sonochemistry. Most of these uses depend on the generation of acoustic cavitation in liquid media but this text, while underlining the importance of the physics and mathematics of cavitation, mainly concentrates on applications of the technology.\nAfter an introduction to the topic and some historical back

In [8]:
with open('GSCU Crawling.csv', 'w', encoding='utf8', newline='') as output_file:
    fc = csv.DictWriter(output_file,
                       fieldnames=allpapers[0].keys(),
                       )
    fc.writeheader()
    fc.writerows(allpapers)